# Assignment 5:
<h2>Question :</h2>
Design a Naïve Bayes classifier for filtering Spam and Ham (Normal) messages from your mail box.. Make a comparative study on the performance of all the three models of Naïve Bayes classifiers. The SMS data set, together with a readme file from UCI Machine Learning Repository, has been provided for your ready reference in the appropriate link on my website.
<br>
Hints: 
1. you may go through this 11 min video: <br>
https://www.youtube.com/watch?v=99MN-rl8jGY <br>
2. However, You are allowed to use only Python NumPy.

In [160]:
#loading required package
import numpy as np

### Loading data set from csv file

In [161]:
#reading data set
read_data = np.loadtxt("spam ham data set.csv",encoding="latin-1" ,dtype=np.str,delimiter="\t")
read_data = read_data[2:]        #removing top 2 useless rows of dataset

## Cleaning dataset
<ul>
    <li>1.convert sms massages to lower case.</li>
    <li>2.remove all punchuations.</li>
    <li>3.split msg to list and remove words with length smaller than 3 from list</li>
</ul>

In [162]:
#cleaning text
import string        #package to get puchuation list(no other use has been done of this package)
exclist = string.punctuation + string.digits           # contain puchuation !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789
y_msg=[]
x_msg=[]

for msg in read_data:
    msg = msg.lower()
    if msg[0]=='h':                #sepearating ham and spam, setting ham=0 and spam=1
        msg = msg[4:]
        y_msg.append(0)
    elif msg[0]=='s':
        msg = msg[3:]
        y_msg.append(1)

    for s in exclist:            #removing punchuation
         msg = msg.replace(s, '')
    msg = msg.replace('\W+', '').replace('\s+', ' ').strip()
    msg = msg.split()            #splitting msg

    to_remove = []
    for m in msg:                 #removing words of length less than 2( a , if ,ok ,is ,etc)
        if len(m)<=2:
            to_remove.append(m)
    for r in to_remove:
        msg.remove(r)
        
    x_msg.append(msg)


### Creating Vacobulary from input dataset

In [163]:
#creating vacobulary
vacob = []
for msg in x_msg:
    for word in msg:
        if word not in vacob:
            vacob.append(word)
vacob.sort()
print("Length of Vacobulary ->",len(vacob))
print("\nTop 20 elements in vacobulary ->",vacob[:20])

Length of Vacobulary -> 8381

Top 20 elements in vacobulary -> ['aah', 'aaooooright', 'aathilove', 'aathiwhere', 'abbey', 'abdomen', 'abeg', 'abelu', 'aberdeen', 'abi', 'ability', 'abiola', 'abj', 'able', 'abnormally', 'about', 'aboutas', 'above', 'abroad', 'absence']


### Creating vector from vacobulary:
for example : vacobulary = ["Hello", "Dear","How" ,"Money","zone"] \
              $x_j$ = ["Hello","Dear","Hello"] \
              vector($x_j$) = [2,1,0,0,0] 

In [164]:
#converting data to frequecy table which will be used as dataset
m = len(y)
n = len(vacob)
print("Shape of data -> ",m,",",n)

x_data_list = []
for i in range(m):
    x_j = []
    for v in vacob:
        freq = x_msg[i].count(v)
        x_j.append(freq)
    x_data_list.append(x_j)


Shape of data ->  5573 , 8381


In [165]:
#converting all data to numpy for further use
x_data = np.array(x_data_list)
y_data = np.array(y_msg).reshape((m,1))


### Splitting data set, 75 % training and 25% as testing.

In [166]:
#splitting of data into training and testing dataset
m,n=x_data.shape
k = int(m*0.75)
x_train = x_data[:k]
y_train = y_data[:k]
x_test = x_data[k:]
y_test = y_data[k:]
print("Training dataset ->",x_train.shape,y_train.shape)
print("Testing dataset  ->",x_test.shape,y_test.shape)

Training dataset -> (4179, 8381) (4179, 1)
Testing dataset  -> (1394, 8381) (1394, 1)


# Designing Naive Bayes Classifier for filtering spam and ham messages.

## 1.Multivariate Bernoulli Event Model :


\begin{equation}
    \phi_{y^{(i)}=1} = \frac{ \sum_{i=1}^{m} 1\{ y^{(i)} =1\}}{m} \\
    \phi_{y^{(i)}=0} = 1-\phi_{y^{(i)}=1}
\end{equation}

In [167]:
m,n = x_train.shape
phi_1 = np.sum(y_train)/m
phi_0 = 1-phi_1
print("phi_1 ->",phi_1)
print("phi_0 ->",phi_0)

phi_1 -> 0.13519980856664274
phi_0 -> 0.8648001914333573


\begin{equation}
\phi_{j/y=0} = \frac{\sum_{i=1}^{m} 1\{ x_j^{(i)}=1 \wedge y^{(i)} =0\} }{ \sum_{i=1}^{m} 1\{ y^{(i)} =0\} }
\end{equation}
\begin{equation}
\phi_{j/y=1} = \frac{\sum_{i=1}^{m} 1\{ x_j^{(i)}=1 \wedge y^{(i)} =1\} }{ \sum_{i=1}^{m} 1\{ y^{(i)} =1\} }
\end{equation}

In [179]:
phi_j_y0 = []
phi_j_y1 = []
sum_y1 = np.sum(y_train)
sum_y0 = m-sum_y1

for j in range(n):
    count_y0=0
    count_y1=0
    for i in range(m):
        if y_train[i]==0 and x_train[i][j]>=1:
            count_y0 += 1
        elif y_train[i]==1 and x_train[i][j]>=1:
            count_y1 += 1
    phi_j_y0.append(count_y0/sum_y0)
    phi_j_y1.append(count_y1/sum_y1)
            

### Defining function to predict and compute Accuracy :
\begin{equation}
P(y_{=1}/x_j) = \frac{P(x_j/y)P(y)}{P(x_j)} \\
prediction = argmax_y(P(y/x))
\end{equation}

In [204]:
#function to predict result
def predict(x):
    p_0=phi_0
    p_1=phi_1
    for j in range(x.shape[0]):
        if x[j]==1:
            px_y0 = phi_j_y0[j]
            px_y1 = phi_j_y1[j]
            if px_y0==0:
                px_y0=0.000000000001 #cases where px_y= 0
            if px_y1==0:
                px_y1=0.000000000001
                
            p_0 = p_0*px_y0
            p_1 = p_1*px_y1
    
    if p_0 < p_1:   #argmax
        return 1
    else:
        return 0
            
    print(p_0,p_1)

#function to compute accuracy 
def accuracy(x,y):
    count = 0
    total = x.shape[0]
    for i in range(x.shape[0]):
        p = predict(x[i])
        if p==y[i]:
            count += 1
    
    return (count*100)/total



In [205]:
#Preciting some results :
# 0 = ham
# 1 = spam
print("0 ----> HAM")
print("1 ----> SPAM")
for i in range(10):
    print("Actual ->",y_test[i],"   Precited -->",predict(x_test[i]))

0 ----> HAM
1 ----> SPAM
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 0
Actual -> [1]    Precited --> 1
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 1
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 0


### Accuracy of Multivariate bernoulli event model :

In [203]:
print("Accuracy : ",accuracy(x_test,y_test),"%")

Accuracy :  97.05882352941177 %


# 2.Multinomial Event Model :

### Calculating $\phi_{y^{(i)}=0}$ , $\phi_{y^{(i)}=1}$

\begin{equation}
\phi_{y^{(i)}=0} = \frac{\sum_{i=1}^{m} \{ y^{(i)}=0 \}}{m} \\
\phi_{y^{(i)}=1} = \frac{\sum_{i=1}^{m} \{ y^{(i)}=1 \}}{m} 
\end{equation}

In [243]:
phi_y1 = np.sum(y_train)/y_train.shape[0]
phi_y0 = 1- phi_y1
print("phi_y0 ----> ",phi_y0)
print("phi_y1 ----> ",phi_y1)

phi_y0 ---->  0.8648001914333573
phi_y1 ---->  0.13519980856664274


### Calculating $\phi_{x/y^{(i)}=0}$ , $\phi_{x/y^{(i)}=1}$

In [250]:
m,n = x_train.shape
sum_x0 = np.zeros(n)
sum_x1 = np.zeros(n)
for i in range(m):
    if y[i]==1:
        sum_x1 += x_train[i]
    else:
        sum_x0 += x_train[i]
        
num_words0 = np.sum(sum_x0)
num_words1 = np.sum(sum_x1)

phi_x_y0 = sum_x0/num_words0
phi_x_y1 = sum_x1/num_words1

[2.81080473e-05 2.81080473e-05 8.43241420e-05 ... 2.81080473e-05
 5.62160947e-05 0.00000000e+00]


### Defining function to predict and calculate accuracy.
\begin{equation}
    Precition = argmax_y P(y/x) \\
    P(y/x) = \frac{P(x/y)P(y)}{P(x)}
\end{equation}

In [254]:
def predict_multinomial(x):
    p_0=phi_y0
    p_1=phi_y0
    for j in range(x.shape[0]):
        if x[j]==1:
            px_y0 = phi_x_y0[j]
            px_y1 = phi_x_y1[j]
            if px_y0==0:
                px_y0=0.000000000001 #cases where px_y= 0
            if px_y1==0:
                px_y1=0.000000000001
                
            p_0 = p_0*px_y0
            p_1 = p_1*px_y1
    
    if p_0 < p_1:   #argmax
        return 1
    else:
        return 0
            
    print(p_0,p_1)

#function to compute accuracy 
def accuracy_multinomial(x,y):
    count = 0
    total = x.shape[0]
    for i in range(x.shape[0]):
        p = predict_multinomial(x[i])
        if p==y[i]:
            count += 1
    
    return (count*100)/total

In [255]:
#Preciting some results :
# 0 = ham
# 1 = spam
print("0 ----> HAM")
print("1 ----> SPAM")
for i in range(10):
    print("Actual ->",y_test[i],"   Precited -->",predict_multinomial(x_test[i]))

0 ----> HAM
1 ----> SPAM
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 0
Actual -> [1]    Precited --> 1
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 1
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 0
Actual -> [0]    Precited --> 0


### Accuracy using Multinomial envent Model:

In [256]:
print("Accuracy : ",accuracy_multinomial(x_test,y_test),"%")

Accuracy :  96.91535150645625 %


# 3.Guassian Discriminant Analysis Model :

### Calculating $\phi$ using the formula :
\begin{equation}
    \phi = \frac{1}{m} \sum _{i=1}^{i=m} y^{(i)}
\end{equation}

In [211]:
def calculate_phi(y):
    m,n = y.shape
    return np.sum(y)/m

print("->",calculate_phi(y_train))

-> 0.13519980856664274


### Calculating $\mu_0$ , $\mu_1$ using the formula :
\begin{equation}
    \mu_0 = \frac{\sum_{i=1}^{i=m}1(y^{(i)}=0)x^{(i)}}{\sum_{i=1}^{i=m}1(y^{(i)}=0)} \\
\end{equation}
\begin{equation}
    \mu_1 = \frac{\sum_{i=1}^{i=m}1(y^{(i)}=1)x^{(i)}}{\sum_{i=1}^{i=m}1(y^{(i)}=1)} 
\end{equation}

In [224]:
def calculate_mu0(x,y):
    m,n = x.shape
    num=np.zeros((1,n))
    den=0
    for i in range(m):
        if y[i]==0:
            num += x[i]
            den += 1
    #print(num)
    return num/den

def calculate_mu1(x,y):
    m,n = x.shape
    num=np.zeros((1,n))
    den=0
    for i in range(m):
        if y[i]==1:
            num += x[i]
            den += 1
    return num/den

print("u0,u1 ->")
print(calculate_mu0(x_train,y_train))
print(calculate_mu1(x_train,y_train))


u0,u1 ->
[[0.0002767  0.0002767  0.00083011 ... 0.0002767  0.0005534  0.        ]]
[[0. 0. 0. ... 0. 0. 0.]]


### Calculate $\sum$ using the formula:
\begin{equation}
    \sum = \frac{1}{m} \sum _{i=1}^{i=m} (x-\mu_{y_{(i)}}). (x-\mu_{y_{(i)}})^T
\end{equation}

In [231]:
def calculate_sigma(x,y,mu_0,mu_1):
    m,n = x.shape
    sigma = np.zeros((n,n))
    for i  in range(m):
        x_i = x[i]
        y_i = y[i]
        if y_i == 1:
            k = (x_i-mu_1).T
        else:
            k = (x_i-mu_0).T
        sigma += k*k.T
    sigma = sigma/m
    return sigma

print("sigma :")
sigma = calculate_sigma(x_train,y_train,calculate_mu0(x_train,y_train),calculate_mu1(x_train,y_train))

sigma :


### Calculating class prior ,i.e class prior :P(y)
\begin{equation}
    P(y^{(i)};\phi) = \phi^{y^{(i)}} (1-\phi)^{1-y^{(i)}}
\end{equation}

In [232]:
def calculate_py(y_i,phi):
    if y_i==1:
        return phi
    else:
        return 1-phi

In [233]:
#using above function to find all required values
phi  = calculate_phi(y_train)
mu_0 = calculate_mu0(x_train,y_train)
mu_1 = calculate_mu1(x_train,y_train)
print("phi ->",phi)
print("mu_0 ->",mu_0)
print("mu_1 ->",mu_1)
print("sigma ->\n",sigma)

phi -> 0.13519980856664274
mu_0 -> [[0.0002767  0.0002767  0.00083011 ... 0.0002767  0.0005534  0.        ]]
mu_1 -> [[0. 0. 0. ... 0. 0. 0.]]
sigma ->
 [[ 2.39225484e-04 -6.62124230e-08 -1.98637269e-07 ... -6.62124230e-08
  -1.32424846e-07  0.00000000e+00]
 [-6.62124230e-08  2.39225484e-04 -1.98637269e-07 ... -6.62124230e-08
  -1.32424846e-07  0.00000000e+00]
 [-1.98637269e-07 -1.98637269e-07  7.17279178e-04 ... -1.98637269e-07
  -3.97274538e-07  0.00000000e+00]
 ...
 [-6.62124230e-08 -6.62124230e-08 -1.98637269e-07 ...  2.39225484e-04
  -1.32424846e-07  0.00000000e+00]
 [-1.32424846e-07 -1.32424846e-07 -3.97274538e-07 ... -1.32424846e-07
   4.78318543e-04  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


### Calculating Likelihood of a feature to be in a class ,i.e. P(x/y;$\mu,\sum$) using formula :
\begin{equation}
    P(x/y ; \mu , \sum) = \frac{1}{(2\pi)^{\frac{n}{2}}|\sum|^{\frac{1}{2}}} exp(-\frac{1}{2}(x-\mu)^T \sum^{-1} (x-\mu))
\end{equation}

In [234]:
def calculate_px_y(x,mu,sigma):
    m = x.shape[0]
    x = x.reshape(1,m)
    n = 2 #number of class
    x_u = (x-mu).T
    x_u_t = x_u.T
    det_sigma = np.linalg.det(sigma)
    inv_sigma = np.linalg.inv(sigma)
    np.matmul(np.matmul(x_u_t,inv_sigma),x_u)
    px_y = np.exp(-0.5*np.matmul(np.matmul(x_u_t,inv_sigma),x_u)) /( np.power((2*np.pi),n/2)* np.sqrt(det_sigma))
    return px_y


# Failed
$\sum$ is singular matrix, gives error while compting inverse.

In [ ]:
print(calculate_px_y(x_test[0],mu_1,sigma))
print(calculate_px_y(x_test[0],mu_0,sigma))

### Calculate P(y/x) for prediction:
\begin{equation}
   P(y/x) = \frac{P(x/y)P(y)}{P(x)} \\
   P(x) = constant\ for\ both\ class \\
   Prediction = argmax_y \frac{P(x/y)P(y)}{P(x)}
\end{equation}



In [ ]:
def predict_gda(x,phi,mu_0,mu_1,sigma):
    p0 = calculate_px_y(x,mu_0,sigma)*calculate_py(0,phi)
    p1 = calculate_px_y(x,mu_1,sigma)*calculate_py(1,phi)
    if p0>p1:
        return 0
    else:
        return 1
    
def accuracy_gda(x_test,y_test,phi,mu_0,mu_1,sigma):
    m,n = x_test.shape
    count = 0
    for i in range(m):
        y_predict = predict_gda(x_test[i],phi,mu_0,mu_1,sigma)
        if y_predict==y_test[i]:
            count +=1
    
    return count*100/m

print("Accuracy = ",accuracy_gda(x_test,y_test,phi,mu_0,mu_1,sigma),end=" %")

## Results :
<ul>
    <li>1. Accuracy of Multivariate Bernoulli = 97.058 %</li>
    <li>2. Accuracy of Multinomial event Model =  96.91535 %</li>
    <li>3. GDA failed to predit,because $\sum^{-1}$ cannot be computed as $\sum$ is singular matrix</li>
    </ul>
    

# Analysis :
<ul>
    <li>Simpler and faster if dataset in available in proper format.</li>
    <li>Preprocessing data and cleaning, creating matrix is more complex than desiging predictor.</li>
    <li>Multivariate Bernoulli has better Accuracy compared to Multinomial event model with very less difference.</li>
    <li>GDA is less efficent in terms of time, compting parameter took longer time.</li>
</ul>